### How to use the values set in python-kernel from js "immediately" in the same jupyter-cell

There is a problem of using a value set in python kernel from js kernel.execute() in the cell where that command is called, or in a loaded method from an external module. Either way, the js "doesn't set variables in python" until all the input cells have been run. This could be an issue of "context", "scope", etc, but it's not pure timing as throwing in sleeps or threads doesn't help, it's more likely due to the way ipython executes calls.

This is most important for passing in notebook name and notebook path, which can only be established via js. It's also useful for establishing timing between js-based events (e.g. `save_notebook` and `reload_notebook`) with python-based events (e.g. `merge.give_answer()`, `gitcomm.push_answer()`, etc).

In this notebook, we establish a way to replicate the problem, by setting a variable three times and seeing which one executes "last" by checking it's value. We also show a variety of ways we attempt to fix it. The helper functions are all in `cpr/misc.py`

One possible way to fix in is with the `ipython_blocking` package, but so far, no luck.

Another possible way is to use jupyter's messaging module (most often used for widgets).

Right now, we get around this by setting the notebook name variable when we load the modules in cpr/, specifically in `cpr.nbx`.

In [2]:
import sys, time, os
sys.path.append('../../')
%reload_ext autoreload
%autoreload 2
from cpr import misc

<IPython.core.display.Javascript object>

##### Determine order of execution
Scenario 1: 
 - set before calling `a=0`
 - set within the module call `a=99`
 - get within calling input
 - set within calling input `a=-77`
 - get in next cell
 
Notice how the output of final cell is `a=99` even though in code execution, that should have been the second set.

In [3]:
a = 0

In [4]:
misc.demo_js_scope_a()
print(f'a after call to demo_js_scope: {a}')
a = -77

<IPython.core.display.Javascript object>

a after call to demo_js_scope: 0


In [5]:
a

99

##### Exception blocks js execution
An exception which occurs before cell is over prevents `display(Javascript(js))` from exectuing

In [6]:
b = 0

In [7]:
misc.demo_js_scope_b() # should set b to 99
print(f'b after call : {b}')
b = -77
raise Exception('blocking exception')

<IPython.core.display.Javascript object>

b after call : 0


Exception: blocking exception

In [8]:
b

-77

##### Verify the pattern we think
multiple calls to display() don't help with exception block

In [10]:
c = 0

In [11]:
misc.demo_js_scope_c(66)
print(f'c after call : {c}')
c= 11
misc.demo_js_scope_c(22)
print(f'c after call : {c}')
raise Exception('blocking exception')
misc.demo_js_scope_c(62)
print(f'c after call : {c}')
c

<IPython.core.display.Javascript object>

c after call : 0


<IPython.core.display.Javascript object>

c after call : 11


Exception: blocking exception

In [12]:
c

11

##### Try using the method here:
https://github.com/kafonek/ipython_blocking/blob/master/ipython_blocking/ipython_blocking.py

In [59]:
ip = get_ipython()

In [60]:
ip.kernel.do_one_iteration()

<Future finished result=None>

In [73]:
e = 0

In [74]:
misc.demo_js_scope_e()
print(f'e after call to demo_js_scope: {e}')
e = -77

<IPython.core.display.Javascript object>

e after call to demo_js_scope: 0


In [76]:
e

99

##### Use threading to attempt to bypass this problem
It still doesn't work: careful - if you run it twice, "it works"

In [9]:
import sys, time, os
sys.path.append('../../')
%reload_ext autoreload
%autoreload 2
from cpr import misc

In [10]:
import time
import threading
import queue

In [11]:
d = 0

In [52]:
q = queue.Queue()
q.put(0)

In [53]:
q.get()

0

In [54]:
q.empty()

True

In [58]:
def run_d_99():
    misc.demo_js_scope_q()
    print('d set')
    
def poll_d(q):
    print('starting')
    for i in range(1_000):
        time.sleep(.001)
        if q.empty(): continue
            
        print('q not empty')
        print(q.get())
        
        return

t0 = threading.Thread(target=run_d_99)
t1 = threading.Thread(target=poll_d, args=(q,))

t0.start()
t1.start()

# t0.join()
t1.join()

print('done')

<IPython.core.display.Javascript object>

starting
d set
done


In [42]:
d

99

##### Example Threading

In [36]:
import threading
from random import randint
from time import sleep


def print_number(number):

    # Sleeps a random 1 to 10 seconds
    rand_int_var = randint(1, 10)
    sleep(rand_int_var)
    print( "Thread " + str(number) + " slept for " + str(rand_int_var) + " seconds")

thread_list = []

for i in range(1, 10):

    # Instantiates the thread
    # (i) does not make a sequence, so (i,)
    t = threading.Thread(target=print_number, args=(i,))
    # Sticks the thread in a list so that it remains accessible
    thread_list.append(t)

# Starts threads
for thread in thread_list:
    thread.start()

# This blocks the calling thread until the thread whose join() method is called is terminated.
# From http://docs.python.org/2/library/threading.html#thread-objects
# for thread in thread_list:
#     thread.join()

# Demonstrates that the main process waited for threads to complete
print ("Done")

Done
Thread 2 slept for 1 secondsThread 6 slept for 1 seconds

Thread 3 slept for 3 seconds
Thread 9 slept for 4 seconds
Thread 1 slept for 6 seconds
Thread 8 slept for 8 seconds
Thread 4 slept for 9 seconds
Thread 7 slept for 9 seconds
Thread 5 slept for 10 seconds
